In [1]:
import requests
import geopandas
from geopandas import GeoSeries
from shapely.geometry import Polygon
import os
import fnmatch
import os
import rasterio
import pandas as pd
import geopandas as gpd
import rasterio
from rasterio.windows import Window
from rasterio.mask import mask
import numpy as np
import plotly.graph_objects as go
import dash
import dash_core_components as dcc
import dash_html_components as html

## Retrieve building coordinates

In [2]:
def get_address():
    
    import requests
    import geopandas
    from geopandas import GeoSeries
    from shapely.geometry import Polygon
    
    street = 'Thonetlaan'
    nb = 133
    pc = 2050
    city = 'Antwerpen'
    
    #Ask for user to input address
    nb = input("Enter house number:")
    street = input("Enter street:")
    city = input("Enter city:")
    pc = input("Enter postcode:")
    #Example address
    #Eugeen de Bocklaan 14, 2900 Schoten
    
    global building_address
    building_address = (street, str(nb), city, str(pc))
    building_address = " ".join(building_address)
    
    #Check user adddress match using api
    req = requests.get(f"https://api.basisregisters.dev-vlaanderen.be/v1/adresmatch?gemeentenaam={city}&straatnaam={street}&huisnummer={nb}&postcode={pc}").json()
    
    #Retrieve objectID for users address
    objectId = req["adresMatches"][0]["adresseerbareObjecten"][0]["objectId"]
    
    #Get building geometry
    req = requests.get(f"https://api.basisregisters.dev-vlaanderen.be/v1/gebouweenheden/{objectId}").json()

    objectId = req["gebouw"]["objectId"]

    req = requests.get(f"https://api.basisregisters.dev-vlaanderen.be/v1/gebouwen/{objectId}").json()

    #Get building polygon coordinates
    global polygon
    polygon = [req["geometriePolygoon"]["polygon"]]
    
    #Convert polygon to more useful geopanda series
    t = []
    
    #Get coordinates
    for i in polygon[0]['coordinates'][0]:
        t.append(tuple(i))
    
    #Convert coordinates to Polygon format
    global p
    p = Polygon(t)
    
    #Save Polygon in geopanda series
    global g
    g = GeoSeries([p])
    
    #Get area of building
    global a
    #Area of the building
    a = g.area

    return p, a

#get_address()

## Create list of all DSM and DTM raster files

In [13]:
def raster_list():
    #Create list of all raster files
    import os
    import fnmatch
    path = os.path.abspath('/media/becode/GOPRO2/1GEOTIFF')
    folder_path = path
    
    global dsmlist, dtmlist
    dsmlist = []
    dtmlist = []


    for path, dirs, files in os.walk(folder_path):
            for filename in files:
                if fnmatch.fnmatch(filename, '*DSM*'):
                    dsmlist.append(filename)
                if fnmatch.fnmatch(filename, '*DTM*'):
                    dtmlist.append(filename)
            print("There are",len(dsmlist), "DSM raster files.")
            print("There are",len(dtmlist), "DTM raster files.")
            
raster_list()

There are 43 DSM raster files.
There are 43 DTM raster files.


## Get bounds of all raster files in Flanders

In [14]:
def get_bounds():

    #Get bounds of rasters
    import os
    import rasterio
    import pandas as pd

    #Set path
    path = os.path.abspath('/media/becode/GOPRO2/1GEOTIFF')

    #Make an empty lists
    dsmbounds = []
    filenames = []

    for f in dsmlist:
    
        filepath = os.path.join(path, f)
    
        #Add raster filename to list
        filenames.append(f)
    
        #Open raster and add bounds to list
        with rasterio.open(filepath) as src:
            dsmbounds.append(src.bounds)
    
    #Create df of filenames and bounds
    global dsm_df
    dsm_df = pd.DataFrame({'filelist': filenames,
                   'bounds': dsmbounds})
 
    #Write df to a file
    dsm_df.to_csv("dsmbounds.csv", index=False)

    #Make an empty lists
    dtmbounds = []
    filenames = []
    
    for f in dtmlist:
    
        filepath = os.path.join(path, f)
    
        #Add raster filename to list
        filenames.append(f)
    
        #Open raster and add bounds to list
        with rasterio.open(filepath) as src:
            dtmbounds.append(src.bounds)

    #Create df of filenames and bounds
    global dtm_df
    dtm_df = pd.DataFrame({'filelist': filenames,
                   'bounds': dtmbounds})
 
    #Write df to a file
    dtm_df.to_csv("dtmbounds.csv", index=False)
    
get_bounds()

In [15]:
dsm_df

,filelist,bounds
0,DHMVIIDSMRAS1m_k15.tif,"(130000.0, 198000.0, 162000.0, 218000.0)"
1,DHMVIIDSMRAS1m_k01.tif,"(145000.0, 238000.0, 162000.0, 247000.0)"
2,DHMVIIDSMRAS1m_k02.tif,"(162000.0, 238000.0, 194000.0, 250000.0)"
3,DHMVIIDSMRAS1m_k03.tif,"(194000.0, 238000.0, 206000.0, 248000.0)"
4,DHMVIIDSMRAS1m_k04.tif,"(53000.0, 218000.0, 66000.0, 225500.0)"
5,DHMVIIDSMRAS1m_k05.tif,"(66000.0, 218000.0, 98000.0, 232000.0)"
6,DHMVIIDSMRAS1m_k06.tif,"(98000.0, 218000.0, 130000.0, 227000.0)"
7,DHMVIIDSMRAS1m_k13.tif,"(66000.0, 198000.0, 98000.0, 218000.0)"
8,DHMVIIDSMRAS1m_k09.tif,"(194000.0, 218000.0, 226000.0, 238000.0)"
9,DHMVIIDSMRAS1m_k11.tif,"(18500.0, 198000.0, 34000.0, 206500.0)"


## Search for raster file that contains the user address

In [5]:
def raster_overlap():

    import geopandas as gpd
    import rasterio
    
    ##Get filenames of overlapping rasters
    dsmlist = []
    dtmlist = []
    
    #Open DSM and DTM raster bounds as df
    #dsm_df = gpd.read_file('dsmbounds.csv')
    #dtm_df = gpd.read_file('dtmbounds.csv')
    #reading bounds from csv gives an error
    
    #Check which DSM rasters overlap the building boundary box
    for i in dsm_df['bounds']:
        dsmlist.append(rasterio.coords.disjoint_bounds(i, p.bounds))

    #Reverse the booleen
    boo = [not bool for bool in dsmlist]

    #Keep the dsm filename
    global dsmfile
    dsmfile = dsm_df.filelist[boo].item()

    #Check which DTM rasters overlap the building boundary box
    for i in dtm_df['bounds']:
        dtmlist.append(rasterio.coords.disjoint_bounds(i, p.bounds))

    #Reverse the booleen
    boo = [not bool for bool in dsmlist]

    #Keep the dtm filename
    global dtmfile
    dtmfile = dtm_df.filelist[boo].item()

    return dsmfile, dtmfile

#raster_overlap()

## Calculate Digital Elevation Model for building

In [6]:
def calculate_dem():
    
    import os
    import rasterio
    from rasterio.windows import Window
    from rasterio.mask import mask
    import numpy as np
    import plotly.graph_objects as go

    #File paths required
    path = os.path.abspath('/media/becode/GOPRO2/1GEOTIFF')
    dsmpath = os.path.join(path, str(dsmfile))
    dtmpath = os.path.join(path, str(dtmfile))

    #Open DSM raster with mask of building shape
    with rasterio.open(dsmpath) as src:
        mask, out_transform, win = rasterio.mask.raster_geometry_mask(dataset=src, shapes=g, invert=False, crop=True, pad=False)
        #Read only pixels within the window/bounds of the building shape
        dsm = src.read(1, window = win)

    #Open DTM raster with mask of building shape
    with rasterio.open(dtmpath) as src:
        mask, out_transform, win = rasterio.mask.raster_geometry_mask(dataset=src, shapes=g, invert=False, crop=True, pad=False)
        #Read only pixels within the window/bounds of the building shape
        dtm = src.read(1, window = win)

    #Calculates raw digital elevation model (no resampling)
    global dem
    dem = dsm - dtm

#calculate_dem()

## Get the height and floor area of the building

In [7]:
def get_height():
    #Show building height and floor area
    height = round(dem.max(), 1)
    print('The building height is:', height, 'meters')

def get_area():
    area = round(int(a), 1)
    print('The building floor area is:', area, 'sq meters')
    
#get_height()
#get_area()

## Plot the building

In [8]:
def plot_building():
    import plotly.graph_objects as go

    #Plot xyz of building
    fig = go.Figure(data=[go.Surface(z=dem)])

    fig.update_layout(title=str(building_address), autosize=False,
                  width=500, height=500,
                  margin=dict(l=65, r=50, b=65, t=90))

    fig.show()

    import dash
    import dash_core_components as dcc
    import dash_html_components as html

    app = dash.Dash()
    app.layout = html.Div([
    dcc.Graph(figure=fig)
    ])

    app.run_server(debug=True, use_reloader=False)

#plot_building()

## Make 3D house

In [9]:
#Make 3D house
get_address()
raster_overlap()
calculate_dem()
get_height()
get_area()
plot_building()

Enter house number:133
Enter street:thonetlaan
Enter city:antwerpen
Enter postcode:2050


NameError: name 'dsm_df' is not defined